In [ ]:
#ames housing dataset_rev2
#to-do in this revision
#1. As learnt from previous attempt,this time need to join test & train data right in the beginning to avoid mismatch
# in no. of columns after OHC
#2. Explore advanced regression techniques such as XGBoost, adaboost

In [1]:
import numpy as np
import pandas as pd
import string 
import matplotlib.pyplot as plt

In [3]:
df_train = pd.read_csv("train_1.csv")
df_test = pd.read_csv("test_1.csv")

In [28]:
df_train.shape

(1460, 80)

In [29]:
df_test.shape

(1459, 80)

In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
output_var=df_train.SalePrice    #copying output variable in a separate variable before dropping it from df_train
df_train=df_train.drop(columns='SalePrice',axis=1)

AttributeError: 'DataFrame' object has no attribute 'SalePrice'

In [13]:
#joining the df_train and df_test row-wise
df=[df_train,df_test]
df_joint=pd.concat(df)
df_joint.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [15]:
#need to check if data has been actually appended or not
print("Dimensions of the newly joint dataset are {}".format(df_joint.shape))

Dimensions of the newly joint dataset are (2919, 80)


In [16]:
df_joint.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2918.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,472.874572,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,215.394815,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,320.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,480.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1488.000000,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [17]:
#counting columnwise NA
null_columns=df_joint.columns[df_joint.isnull().any()]
df_joint[null_columns].isnull().sum()

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [18]:
df_joint_1 = df_joint.drop(['MiscFeature','Alley','PoolQC','Id'],axis=1) #dropping columns with >90% NULLS

In [20]:
#checking new dimensions of the dataset after dropping
print("Dimesnsions of the df_joint_1 are {}".format(df_joint_1.shape))

Dimesnsions of the df_joint_1 are (2919, 76)


In [21]:
#imputation. df_joint will be divided into df_joint_n containing only numeric data & df_joint_c containing only categorical data
from sklearn.impute import SimpleImputer
df_joint_2 = df_joint_1.copy()
df_joint_n=df_joint_2.select_dtypes(exclude=['object']) #excluding categorical to retain numeric data

In [22]:
#numerical imputation
imp_n = SimpleImputer(missing_values=np.nan, strategy='median') #define the numerical imputer
df_joint_n_i = imp_n.fit_transform(df_joint_n)
columns_numerical = df_joint_n.columns
df_joint_n_i = pd.DataFrame(df_joint_n_i, columns=columns_numerical) #reassigning the cols as imputation loses cols

In [23]:
#--checking if nulls still remain post imputation
null_columns=df_joint_n_i.columns[df_joint_n_i.isnull().any()]
df_joint_n_i[null_columns].isnull().sum()

Series([], dtype: float64)

In [25]:
#categorical imputation
df_joint_c=df_joint_2.drop(columns_numerical,axis=1) #starting categorical imputation
df_joint_c.dtypes #to check the df_joint_c does not have any numeric col left
imp_c = SimpleImputer(missing_values=np.nan, strategy='most_frequent') #define the categorical imputer
df_joint_c_i = imp_c.fit_transform(df_joint_c)
columns_categorical = df_joint_c.columns
df_joint_c_i = pd.DataFrame(df_joint_c_i, columns=columns_categorical)


In [26]:
#now need to merge these 2 dataframes-df_joint_n_i & df_joint_c_i COLUMNWISE to form cleansed dataframe from original df
df_clean=pd.merge(df_joint_n_i,df_joint_c_i,on=df_joint_n_i.index)
df_clean=df_clean.drop(['key_0'],axis=1)
df_clean.shape

(2919, 76)

In [27]:
#OHC
my_imputer=SimpleImputer()
ohc_df_clean = pd.get_dummies(df_clean) #pd.getdummies automatically assigns colnames to newcols
ohc_df_clean.shape

(2919, 279)

In [48]:
#improving on accuracy by doing dim reduction - GINI importance
# Print the name and GINI IMPORTANCE of each feature
feat_labels = ohc_df_clean.columns  
#print(feat_labels)
temp_1 = []             #storing in list as I find sorting lists easy :)
for feature in zip(feat_labels, rf.feature_importances_):
    #print(feature)
    temp_1.append(feature)


temp_1.sort(key=lambda x:x[1],reverse=True)
print(temp_1)

[('OverallQual', 0.5932863411864429), ('GrLivArea', 0.1094654143248909), ('1stFlrSF', 0.03757219033385364), ('BsmtFinSF1', 0.03617047773723413), ('TotalBsmtSF', 0.032457403186556946), ('GarageArea', 0.017418887174503246), ('MasVnrArea', 0.017069270007465352), ('LotArea', 0.01667553597343297), ('FullBath', 0.014156848988605034), ('YearRemodAdd', 0.00846028982002121), ('YearBuilt', 0.007097530953182743), ('MoSold', 0.007027982411329026), ('GarageType_Detchd', 0.005965502544634125), ('2ndFlrSF', 0.005331561664682997), ('LotFrontage', 0.004932715962286358), ('OverallCond', 0.004683220106447453), ('BsmtUnfSF', 0.004678107958897139), ('KitchenQual_Gd', 0.0035392574070707444), ('HalfBath', 0.0034832243032811164), ('Fireplaces', 0.003430934323798798), ('OpenPorchSF', 0.0032197338214391565), ('LandContour_Bnk', 0.00315692064310136), ('GarageYrBlt', 0.0029099206053888145), ('GarageFinish_Unf', 0.00277996805293968), ('BsmtQual_Gd', 0.00258048460577889), ('YrSold', 0.002515338162118722), ('WoodDec

In [49]:
feat_sel_col_drop=(['Neighborhood_IDOTRR','FireplaceQu_Po','GarageType_BuiltIn','GarageQual_Gd','LotShape_IR2','Functional_Typ','GarageCond_TA','HeatingQC_Fa','GarageFinish_RFn','Foundation_BrkTil','FireplaceQu_Ex','LowQualFinSF','Fence_GdWo','BsmtFinType1_Rec','Neighborhood_NWAmes','Exterior2nd_CmentBd','SaleType_WD','Exterior2nd_HdBoard','BldgType_1Fam','BsmtCond_TA','Exterior1st_Plywood','Neighborhood_Edwards','Exterior2nd_MetalSd','Condition1_Artery','GarageType_2Types','Neighborhood_Timber','Electrical_FuseF','BldgType_2fmCon','Electrical_FuseA','SaleCondition_Partial','Exterior1st_CemntBd','Functional_Maj1','BsmtFinType1_LwQ','Neighborhood_NridgHt','BsmtQual_Fa','BldgType_TwnhsE','Condition1_RRAn','MSZoning_FV','SaleType_ConLD','MasVnrType_BrkCmn','Foundation_Wood','GarageCond_Fa','HouseStyle_1.5Fin','SaleType_ConLw','BsmtFinType2_BLQ','MiscVal','SaleCondition_Family','Neighborhood_BrkSide','LotConfig_FR2','KitchenQual_Fa','Neighborhood_Mitchel','PavedDrive_P','Condition1_PosA','Exterior2nd_Wd Shng','Functional_Maj2','BsmtHalfBath','HouseStyle_SFoyer','GarageCond_Gd','Neighborhood_OldTown','SaleType_ConLI','Exterior1st_AsphShn','RoofMatl_Tar&Grv','ExterCond_Ex','Exterior1st_HdBoard','Heating_Grav','BsmtFinType2_Unf','MSZoning_RH','ExterCond_Gd','Neighborhood_Blueste','KitchenAbvGr','Fence_GdPrv','Functional_Min1','Exterior2nd_Stucco','Exterior2nd_Stone','RoofStyle_Gambrel','GarageType_Basment','GarageType_CarPort','BsmtFinType2_GLQ','HouseStyle_SLvl','Neighborhood_ClearCr','BldgType_Duplex','BsmtFinType2_LwQ','Condition1_RRAe','Exterior2nd_AsbShng','LandSlope_Sev','GarageQual_Fa','ExterQual_Ex','Street_Grvl','Functional_Min2','RoofStyle_Mansard','Functional_Mod','Heating_Wall','Heating_GasW','SaleCondition_Alloca','GarageQual_Po','Electrical_FuseP','SaleType_Con','Heating_GasA','FireplaceQu_Fa','Exterior2nd_ImStucc','Condition1_RRNn','RoofMatl_WdShake','Exterior1st_Stucco','LotShape_IR3','Exterior1st_WdShing','HouseStyle_1.5Unf','Neighborhood_NPkVill','SaleType_CWD','HouseStyle_2.5Fin','Foundation_Stone','Neighborhood_Gilbert','Utilities_AllPub','LotConfig_FR3','Neighborhood_Blmngtn','HouseStyle_2.5Unf','Condition1_RRNe','BsmtFinType2_ALQ','ExterQual_Fa','Condition1_PosN','PoolArea','Street_Pave','Utilities_NoSeWa','Neighborhood_BrDale','Neighborhood_MeadowV','Condition2_Artery','Condition2_Feedr','Condition2_Norm','Condition2_PosA','Condition2_PosN','Condition2_RRAe','Condition2_RRAn','Condition2_RRNn','RoofStyle_Shed','RoofMatl_ClyTile','RoofMatl_Membran','RoofMatl_Metal','RoofMatl_Roll','Exterior1st_AsbShng','Exterior1st_BrkComm','Exterior1st_CBlock','Exterior1st_ImStucc','Exterior1st_Stone','Exterior2nd_AsphShn','Exterior2nd_Brk Cmn','Exterior2nd_CBlock','Exterior2nd_Other','ExterCond_Po','Foundation_Slab','BsmtCond_Po','Heating_Floor','Heating_OthW','HeatingQC_Po','Electrical_Mix','Functional_Sev','GarageQual_Ex','GarageCond_Ex','GarageCond_Po','Fence_MnWw','SaleType_Oth','SaleCondition_AdjLand'])
ohc_df_clean_gini=ohc_df_clean.drop(columns=feat_sel_col_drop)
ohc_df_clean_gini.shape #(2919,119) 160 columns dropped

(2919, 119)

In [50]:
#now separate original test & train data so that k-fold cross validation can run only on test data
# From df_train.shape we know first 1460 rows are train data
df_train_1=ohc_df_clean_gini[0:1460]
df_train_1=df_train_1.join(output_var) #joining the output variable SalePrice back into the train dataframe

In [51]:
df_train_1.shape

(1460, 120)

In [52]:
df_test_1=ohc_df_clean_gini[1460:2919]
df_test_1.shape

(1459, 119)

In [56]:
#k-fold cross validation part1
from sklearn.model_selection import train_test_split
all_X=df_train_1.drop(['SalePrice'],axis=1)
#print(all_X)
all_y=df_train_1.SalePrice
#print(all_y)
train_X, test_X, train_y, test_y = train_test_split(all_X, all_y, test_size=0.20,random_state=0)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_X,train_y)
prediction=rf.predict(test_X)
predict=pd.Series(prediction)


C:\Users\eajamat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [57]:
#k-fold cross validation part2
from sklearn.metrics import r2_score
r2score=r2_score(test_y,predict)
print(r2score) #0.75 - badi kam hai bc - raised to 0.83 after dim red using GINI then changed to 0.79

0.7949735470416215


In [61]:
#our final datasets :
#df_train_1 (1460,120)
#df_test_1(1459,119)
rf.fit(all_X,all_y)                     #train_X,train_y
prediction_final=rf.predict(df_test_1) #test_X
predict_final=pd.Series(prediction_final)
print(predict_final)

0       133510.0
1       161100.0
2       180950.0
3       171690.0
4       193800.0
5       183150.0
6       172153.4
7       173500.0
8       177379.5
9       114840.0
10      193980.0
11       92790.0
12       92850.0
13      152040.0
14      154280.0
15      408880.0
16      259826.8
17      304383.2
18      240491.9
19      448094.2
20      315456.1
21      222603.5
22      171911.7
23      177848.9
24      172666.5
25      198500.0
26      307639.6
27      241289.0
28      204655.6
29      196040.0
          ...   
1429    101300.0
1430    144230.0
1431     72877.2
1432    114893.8
1433     59601.1
1434    337019.6
1435    314000.0
1436    188290.0
1437    141525.8
1438    244606.1
1439    155535.0
1440    194090.0
1441    195500.0
1442    353222.6
1443    354471.8
1444     99700.0
1445    201159.6
1446    112800.0
1447    126890.0
1448    153398.5
1449     83700.0
1450     85700.0
1451    154195.0
1452     89650.0
1453     74990.0
1454     82850.0
1455     85000.0
1456    156820

In [63]:
#kaggle submission
submission = pd.DataFrame({'Id':df_test['Id'],'SalePrice':predict_final})

#Visualize the first 5 rows
submission.head()

,Id,SalePrice
0,1461,133510.0
1,1462,161100.0
2,1463,180950.0
3,1464,171690.0
4,1465,193800.0


In [64]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'ames_housing_dec.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: ames_housing_dec.csv
